In [1]:
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
import pickle
import os
import csv 
import pandas as pd
import statistics as stat
import numpy as np
import math
import joblib

dtTrain = pickle.load( open( "pickle/train.pkl", "rb" ) )
dtTrain

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
25,0,-0.3792,0.143793,95.7257,46.054309,17,19,37,24,13,...,0.213900,0.589992,0.550366,0.180541,19.195738,66.593446,104.821981,0.654636,1.320899,5.638
9,0,-2.1104,4.453788,129.7840,69.909411,17,19,60,33,27,...,0.315636,0.627324,0.427534,0.372554,28.776159,198.480482,405.367508,0.471712,1.427413,7.086
20,0,-0.8571,0.734620,115.9860,60.628653,17,19,51,30,21,...,0.162506,0.562672,0.442445,0.346526,30.841801,149.876581,283.053745,0.716541,1.351643,7.921
18,0,-0.4120,0.169744,121.2063,61.430653,17,19,52,31,21,...,0.217150,0.581178,0.446674,0.292804,29.122886,160.182183,290.497609,0.636938,1.320656,7.921
67,0,0.4644,0.215667,129.1337,65.827032,16,18,55,31,24,...,0.136583,0.560517,0.334018,0.472880,32.007329,155.151568,337.271370,0.734012,1.367415,7.046
19,0,-1.1451,1.311254,118.8976,63.722239,17,19,54,31,23,...,0.409393,0.576481,0.466130,0.445640,22.843184,141.771169,310.524239,0.416118,1.488252,7.921
21,0,0.1839,0.033819,94.1202,45.252309,17,19,36,23,13,...,0.213878,0.572056,0.535908,0.305180,18.129260,60.447177,98.686647,0.648262,1.413145,6.076
23,0,0.2952,0.087143,103.2506,51.439481,17,19,42,25,17,...,0.164189,0.589782,0.521981,0.374881,23.417730,85.946027,150.465743,0.717698,1.486644,6.194
90,0,-0.9412,0.885857,138.5674,67.143860,23,25,55,35,20,...,0.129826,0.557024,0.393904,0.330701,39.856182,225.601952,504.233052,0.753101,1.281629,8.155
62,0,-0.0491,0.002411,117.6457,58.675067,16,18,48,29,19,...,0.138479,0.504341,0.393461,0.387005,28.908483,122.072643,240.538164,0.743858,1.284807,7.638


In [2]:
def similarValue(dframe):
    colList = [] #List for column
    # colDframe = dframe.columns.values #get all columns
    for j in range(0,dframe.shape[1]):
        topEl = stat.mode(dframe.iloc[:,j]) # Get modus value from j column
        duplicate = 0
        # Loop comparing each row with modus
        for i in range(0,74):
            comp = dframe.iloc[i,j]
            if comp == topEl:
                duplicate += 1
        if duplicate >= 37 :
            colList.append(j)    
    return colList

def stdDeviation(dframe):
    stdevList = []
    # colDframe = dframe.columns.values
    for j in range(0,dframe.shape[1]-1):
        # Loop search standard deviation each column
        stddev = 0
        sample = dframe.iloc[:,j] #Get all row in j column for standard deviation
        stddev = np.std(sample) #Get standard deviation from 'sample' list
        if stddev <= 0.95 :
            # print(j, "PICKED")
            stdevList.append(j)
    return stdevList

def checkNa(dframe):
    naList = []
    # colDframe = dframe.columns.values
    for j in range(0,dframe.shape[1]):
        cknan = dframe.iloc[0,j]
        if math.isnan(cknan): #Check NaN value at row 0 in column j
            naList.append(j)
    return naList

def checkInf(dframe):
    infList = []
    colDframe = dframe.columns.values
    for j in range(0,len(colDframe)):
        cknan = dframe.iloc[0,j]
        if math.isinf(cknan): #Check Infinite value at row 0 in column j
            infList.append(j)
    return infList

def checkDupLabel(dframe):
    dupLabel = []
    for i in range(0,dframe.shape[1]):
        label = dframe.columns.values[i]
        for j in range(i+1, dframe.shape[1]):
            if(label == dframe.columns.values[j]):
                print(0)
    return dupLabel

# def lowCorrP(dframe):
#     label_ = dframe.columns.values
#     pic50 = dframe.iloc[:,-1]
#     corr_y = [pearsonr(dframe.iloc[:,i],pic50) for i in range(dframe.shape[1])]
#     corr_y = [np.abs(corr_y[i][0]) for i in range(dframe.shape[1])]
#     corr_lim = 0.20
#     hi_corr = []
#     for i in range(len(corr_y)):
#         if corr_y[i] > corr_lim:
#             hi_corr.append(i)
# #     label_idx = label_[hi_corr]
# #     dtLowCorr = dframe.loc[:,label_idx]
#     # print(X_train)
#     print("descriptor number after removing low correlation with target: {}".format(dtLowCorr.shape[1]))
#     return dtLowCorr

def lowCorrPMod(dframe):
    label_ = dframe.columns.values
    pic50 = dframe.iloc[:,-1]
    corr_y = [pearsonr(dframe.iloc[:,i],pic50) for i in range(dframe.shape[1])]
    corr_y = [np.abs(corr_y[i][0]) for i in range(dframe.shape[1])]
    corr_lim = 0.20
    hi_corr = []
    for i in range(len(corr_y)):
        if corr_y[i] > corr_lim:
            hi_corr.append(i)
#     label_idx = label_[hi_corr]
#     dtLowCorr = dframe.loc[:,label_idx]
    # print(X_train)
#     print("descriptor number after removing low correlation with target: {}".format(dtLowCorr.shape[1]))
    return corr_y

def highCorrP(dframe):
    # -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
    # re-calculate correlation with pic50
    label_ = dframe.columns.values
    pic50 = dframe.iloc[:,-1]
    corr_y = [pearsonr(dframe.iloc[:,i],pic50) for i in range(dframe.shape[1])]
    corr_y = [np.abs(corr_y[i][0]) for i in range(dframe.shape[1])]
    desc_num = dframe.shape[1]
    # calculate correlation for each descriptor
    corr_matrix = np.corrcoef(dframe.T)
    corr_lim = 0.80
    low_corr = np.arange(desc_num).tolist()
    tmp = np.arange(desc_num).tolist()
    for i in np.arange(desc_num):
        tmp.remove(i)
        for j in tmp:
            corr_ = np.abs(corr_matrix[i,j])
            if corr_ >= corr_lim:
                if corr_y[i] > corr_y[j]:
                    if j in low_corr:
                        low_corr.remove(j)
                else:
                    if i in low_corr:
                        low_corr.remove(i)
    label_idx = label_[low_corr]
    dtHiCorr = dframe.loc[:,label_idx]
    print("descriptor number after removing high correlation descriptor: {}".format(dtHiCorr.shape[1]))
    return dtHiCorr

def highCorrPMod(dframe):
    # -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
    # re-calculate correlation with pic50
    label_ = dframe.columns.values
    pic50 = dframe.iloc[:,-1]
    corr_y = [pearsonr(dframe.iloc[:,i],pic50) for i in range(dframe.shape[1])]
    corr_y = [np.abs(corr_y[i][0]) for i in range(dframe.shape[1])]
    desc_num = dframe.shape[1]
    # calculate correlation for each descriptor
    corr_matrix = np.corrcoef(dframe.T)
    corr_lim = 0.80
    low_corr = np.arange(desc_num).tolist()
    tmp = np.arange(desc_num).tolist()
    for i in np.arange(desc_num):
        tmp.remove(i)
        for j in tmp:
            corr_ = np.abs(corr_matrix[i,j])
            if corr_ >= corr_lim:
                if corr_y[i] > corr_y[j]:
                    if j in low_corr:
                        low_corr.remove(j)
                else:
                    if i in low_corr:
                        low_corr.remove(i)
    label_idx = label_[low_corr]
    dtHiCorr = dframe.loc[:,label_idx]
    print("descriptor number after removing high correlation descriptor: {}".format(dtHiCorr.shape[1]))
    return corr_y

def dropColumn(dframe, tList):
    dropped = pd.DataFrame() #Create empty dataframe
    dropped = dframe.drop(dframe.columns[tList], axis=1) #drop column based on list by column
    return dropped

In [3]:
dtTrain.shape

(74, 1876)

In [4]:
##### STEP 0 Reduces NAN Values
raw = len(dtTrain.columns)
dtTrain.dropna(axis=1, how='any', thresh=None, subset=None, inplace=True)
print("#0. Trainset ", dtTrain.shape[1] , " | ", raw-dtTrain.shape[1], " NaN value columns removed")

#0. Trainset  1876  |  0  NaN value columns removed


In [10]:
train_corr = dtTrain.corr(method='pearson')
train_corr = train_corr.iloc[:-1,[-1]]
train_corr.sort_values("pIC50", ascending = False, inplace = True)
train_corr = train_corr.iloc[:100,:]
train_corr
# train_corr.to_csv(r'data/data_corr.csv', index= None, header = True)

,pIC50
nHBAcc_Lipinski,0.764790
SM1_Dzi,0.764160
GRAV-2,0.759958
GRAV-3,0.759895
GRAV-1,0.759448
nHBa,0.759151
WTPT-3,0.755751
nHBAcc2,0.750469
GRAV-5,0.750469
GRAV-6,0.750441


In [11]:
train_corr

,pIC50
nHBAcc_Lipinski,0.764790
SM1_Dzi,0.764160
GRAV-2,0.759958
GRAV-3,0.759895
GRAV-1,0.759448
nHBa,0.759151
WTPT-3,0.755751
nHBAcc2,0.750469
GRAV-5,0.750469
GRAV-6,0.750441


In [4]:
corr_100 = train_corr.index.values.tolist()[:100]
corr_100
# joblib.dump(corr_100,'./pickle/corr_100.pkl')

['nHBAcc_Lipinski',
 'SM1_Dzi',
 'GRAV-2',
 'GRAV-3',
 'GRAV-1',
 'nHBa',
 'WTPT-3',
 'nHBAcc2',
 'GRAV-5',
 'GRAV-6',
 'GRAV-4',
 'SM1_Dze',
 'ETA_Eta_F',
 'SpDiam_Dzp',
 'GRAVH-1',
 'sumI',
 'GRAVH-2',
 'GRAVH-3',
 'piPC2',
 'piPC1',
 'DPSA-2',
 'EE_Dzp',
 'SpMax_Dzp',
 'SpAbs_Dzp',
 'SpAD_Dzp',
 'SpMAD_Dzp',
 'WNSA-1',
 'ATS1s',
 'SpDiam_Dzv',
 'ETA_Beta',
 'WPSA-3',
 'ETA_Eta_F_L',
 'ATS2s',
 'ATS1m',
 'ATS6m',
 'MW',
 'ATS7m',
 'EE_Dzv',
 'SpMax_Dzv',
 'SpAbs_Dzv',
 'SPC-4',
 'SpAD_Dzv',
 'ATS2m',
 'ETA_Eta_R',
 'ATSC0c',
 'SP-0',
 'ATSC0e',
 'Kier1',
 'SHBa',
 'SP-2',
 'PPSA-3',
 'WPATH',
 'GGI2',
 'MOMI-X',
 'SpMax_D',
 'EE_D',
 'SpAD_D',
 'GGI1',
 'Kier3',
 'DPSA-3',
 'SpDiam_D',
 'SpMAD_Dzv',
 'nHeavyAtom',
 'SP-3',
 'VR3_Dzs',
 'SpMax6_Bhs',
 'ATS5m',
 'ATS0s',
 'MPC2',
 'Tm',
 'VR3_D',
 'TWC',
 'Zagreb',
 'ATS0m',
 'DELS',
 'SpDiam_Dt',
 'SpMax4_Bhm',
 'WTPT-1',
 'SP-1',
 'ETA_Eta_R_L',
 'SpAD_Dt',
 'SpMax_Dt',
 'EE_Dt',
 'SpMAD_D',
 'SRW4',
 'nBonds',
 'VAdjMat',
 'ECCEN',


In [13]:
###### STEP 3  Low Correlations (to target)
# dtTrain = lowCorrP(dtTrain)
corr_target = lowCorrPMod(dtTrain)
dtTrain
# corr_target
# dtTrainCorr = dtTrain.T
# dtTrainCorr.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
25,0,-0.3792,0.143793,95.7257,46.054309,17,19,37,24,13,...,0.213900,0.589992,0.550366,0.180541,19.195738,66.593446,104.821981,0.654636,1.320899,5.638
9,0,-2.1104,4.453788,129.7840,69.909411,17,19,60,33,27,...,0.315636,0.627324,0.427534,0.372554,28.776159,198.480482,405.367508,0.471712,1.427413,7.086
20,0,-0.8571,0.734620,115.9860,60.628653,17,19,51,30,21,...,0.162506,0.562672,0.442445,0.346526,30.841801,149.876581,283.053745,0.716541,1.351643,7.921
18,0,-0.4120,0.169744,121.2063,61.430653,17,19,52,31,21,...,0.217150,0.581178,0.446674,0.292804,29.122886,160.182183,290.497609,0.636938,1.320656,7.921
67,0,0.4644,0.215667,129.1337,65.827032,16,18,55,31,24,...,0.136583,0.560517,0.334018,0.472880,32.007329,155.151568,337.271370,0.734012,1.367415,7.046
19,0,-1.1451,1.311254,118.8976,63.722239,17,19,54,31,23,...,0.409393,0.576481,0.466130,0.445640,22.843184,141.771169,310.524239,0.416118,1.488252,7.921
21,0,0.1839,0.033819,94.1202,45.252309,17,19,36,23,13,...,0.213878,0.572056,0.535908,0.305180,18.129260,60.447177,98.686647,0.648262,1.413145,6.076
23,0,0.2952,0.087143,103.2506,51.439481,17,19,42,25,17,...,0.164189,0.589782,0.521981,0.374881,23.417730,85.946027,150.465743,0.717698,1.486644,6.194
90,0,-0.9412,0.885857,138.5674,67.143860,23,25,55,35,20,...,0.129826,0.557024,0.393904,0.330701,39.856182,225.601952,504.233052,0.753101,1.281629,8.155
62,0,-0.0491,0.002411,117.6457,58.675067,16,18,48,29,19,...,0.138479,0.504341,0.393461,0.387005,28.908483,122.072643,240.538164,0.743858,1.284807,7.638


In [131]:
dtTrainCorr.shape, len(corr_target)
dtTrainCorr['corr_target'] = corr_target
dtTrainCorr.head()

,25,9,20,18,67,19,21,23,90,62,...,15,68,35,85,61,32,3,30,75,corr_target
nAcid,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
ALogP,-0.379200,-2.110400,-0.857100,-0.412000,0.464400,-1.145100,0.183900,0.295200,-0.941200,-0.049100,...,-0.830600,-0.202700,-0.663600,-1.429300,-0.160000,-1.307100,-0.620900,0.430000,-0.741700,0.105016
ALogp2,0.143793,4.453788,0.734620,0.169744,0.215667,1.311254,0.033819,0.087143,0.885857,0.002411,...,0.689896,0.041087,0.440365,2.042898,0.025600,1.708510,0.385517,0.184900,0.550119,0.192905
AMR,95.725700,129.784000,115.986000,121.206300,129.133700,118.897600,94.120200,103.250600,138.567400,117.645700,...,114.274500,125.925700,139.317300,131.448100,109.875600,130.985000,123.267200,106.938100,148.304600,0.674030
apol,46.054309,69.909411,60.628653,61.430653,65.827032,63.722239,45.252309,51.439481,67.143860,58.675067,...,60.195446,64.500239,72.471411,64.050274,54.786274,69.909411,62.757446,51.245895,73.168239,0.567876


In [9]:
###### STEP 4 High Correlations
dtTrain = highCorrP(dtTrain)

descriptor number after removing high correlation descriptor: 78


In [160]:
dtTrainCorr = dtTrainCorr.sort_values(by='corr_target', ascending=False)
dtTrainCorr.head()

,25,9,20,18,67,19,21,23,90,62,...,15,68,35,85,61,32,3,30,75,corr_target
pIC50,5.638000,7.086000,7.921000,7.921000,7.046000,7.921000,6.076000,6.194000,8.155000,7.638000,...,6.108000,7.337000,6.921000,8.046000,6.824000,7.301000,6.710000,6.468000,7.620000,1.000000
nHBAcc_Lipinski,6.000000,8.000000,8.000000,9.000000,8.000000,8.000000,5.000000,5.000000,10.000000,8.000000,...,7.000000,9.000000,9.000000,10.000000,8.000000,8.000000,8.000000,5.000000,11.000000,0.764790
SM1_Dzi,1.195151,1.645650,1.645650,1.818785,1.610865,1.645650,1.022017,1.022017,2.044034,1.558750,...,1.420401,1.836115,1.818785,2.044034,1.610865,1.645650,1.593535,0.935117,2.165053,0.764160
GRAV-2,46.813167,53.733578,52.416497,53.530352,53.666070,52.996035,45.709411,47.046373,56.654924,52.729939,...,50.721304,54.097163,56.282486,56.254954,50.746442,53.848863,53.049670,48.205039,59.332476,0.759958
GRAV-3,12.989089,14.239580,14.005931,14.203653,14.227651,14.108979,12.784108,13.032191,14.751124,14.061711,...,13.702304,14.303742,14.686406,14.681616,13.706831,14.259940,14.118497,13.245295,15.212303,0.759895


In [161]:
topHundred = dtTrainCorr.iloc[1:101,-1]
topHundredDesc = topHundred.index
topHundredDesc

Index(['nHBAcc_Lipinski', 'SM1_Dzi', 'GRAV-2', 'GRAV-3', 'GRAV-1', 'nHBa',
       'WTPT-3', 'nHBAcc2', 'GRAV-5', 'GRAV-6', 'WNSA-2', 'GRAV-4', 'SM1_Dze',
       'SM1_Dzv', 'PNSA-2', 'ETA_Eta_F', 'SpDiam_Dzp', 'GRAVH-1', 'sumI',
       'GRAVH-2', 'GRAVH-3', 'piPC2', 'piPC1', 'DPSA-2', 'SpMax_Dzp', 'EE_Dzp',
       'SpAbs_Dzp', 'SpAD_Dzp', 'SpMAD_Dzp', 'WNSA-1', 'ATS1s', 'SpDiam_Dzv',
       'ETA_Beta', 'WNSA-3', 'WPSA-3', 'ETA_Eta_F_L', 'ATS2s', 'ATS1m',
       'ATS6m', 'MW', 'RNCG', 'SM1_Dzp', 'ATS7m', 'SpMax_Dzv', 'EE_Dzv',
       'SpAbs_Dzv', 'SPC-4', 'SpAD_Dzv', 'ATS2m', 'ETA_Eta_R', 'ATSC0c',
       'SP-0', 'ATSC0e', 'Kier1', 'SHBa', 'SP-2', 'PPSA-3', 'WPATH', 'GGI2',
       'MOMI-X', 'EE_D', 'SpAD_D', 'SpMax_D', 'GGI1', 'Kier3', 'DPSA-3',
       'SpDiam_D', 'SpMAD_Dzv', 'nHeavyAtom', 'SP-3', 'VR3_Dzs', 'SpMax6_Bhs',
       'ATS5m', 'ATS0s', 'MPC2', 'Tm', 'VR3_D', 'TWC', 'Zagreb', 'ATS0m',
       'DELS', 'SpDiam_Dt', 'SpMax4_Bhm', 'WTPT-1', 'SP-1', 'ETA_Eta_R_L',
       'EE_Dt', 'S

In [162]:
pickle.dump(topHundredDesc, open('./pickle/hundredDesc.pkl', 'wb'))